# Default notebook

This default notebook is executed using Databricks Workflows as defined in resources/bundle.job.yml.

In [ ]:
ctr_catalog_name = dbutils.widgets.get("ctr_catalog_name")
raw_catalog_name = dbutils.widgets.get("raw_catalog_name")
stg_catalog_name = dbutils.widgets.get("stg_catalog_name")
managed_location = dbutils.widgets.get("managed_location")

spark.sql(f"""
    create schema if not exists {ctr_catalog_name}.loading
    managed location '{managed_location}'
""")

spark.sql(f"""
    create table if not exists {ctr_catalog_name}.loading.data_ingestion (
        schema_name string, 
        table_name string,
        primary_key string not null,
        stg_primary_key string,
        active boolean not null,
        filter string,
        selected string,
        constraint data_ingestion_pk primary key (schema_name, table_name)
    )
""")

spark.sql(f"""
    create table if not exists {ctr_catalog_name}.loading.data_ingestion_log (
        catalog_name string,
        schema_name string, 
        table_name string,
        ingestion_date timestamp,
        movements integer not null,
        error string,
        constraint data_ingestion_log_pk primary key (catalog_name, schema_name, table_name, ingestion_date)
    )
""")

In [ ]:
df_schemas = (spark.read
    .table(f"{ctr_catalog_name}.loading.data_ingestion")
    .select("schema_name")
    .distinct()
    .where("active = true")
)

for schema_name in [row["schema_name"] for row in df_schemas.collect()]:
    spark.sql(f"""
        create schema if not exists {raw_catalog_name}.{schema_name}
        managed location '{managed_location}';    
    """)

    spark.sql(f"""
        create schema if not exists {stg_catalog_name}.{schema_name}
        managed location '{managed_location}';    
    """)